In [2]:
import pika;
import json;
import time;

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

In [10]:
MQ_server = '192.168.0.202'
driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
extension_path = 'C:\\chromedriver-win64\\bypass-paywalls-firefox-clean-master.crx'

In [11]:
service = Service(executable_path=driver_path)
option = Options()
option.add_extension(extension=extension_path)
web = webdriver.Chrome(service=service,options=option)

In [12]:
def webGet(url, title):
    web.get(url=url)
    textdata = web.find_elements(By.CLASS_NAME, 'evys1bk0')
    artifact = ''
    for data in textdata:
        artifact = artifact + data.text
    safe_filename = re.sub(r'[<>:"/\\|?*]', '_', title) + ".txt"
    with open(safe_filename,"w") as file:
        file.write(artifact)


In [13]:
connection = pika.BlockingConnection(pika.ConnectionParameters(MQ_server))
channel = connection.channel()
channel.basic_qos(prefetch_count=1)
try:
    while True:
        method_frame, header_frame, body = channel.basic_get(queue='nytime', auto_ack=False)

        if method_frame:
            data = json.loads(body)
            print(f" [x] Received title={data['title']}.\n author={data['author']},\n time = {data['date']}")
            webGet(data['url'], data['title'])
            time.sleep(10)
            channel.basic_ack(delivery_tag=method_frame.delivery_tag)
        else:
            print(" [*] No messages in queue.")
            time.sleep(10)
except KeyboardInterrupt:
    print("\n [*] Exiting.")
finally:
    connection.close()

 [x] Received title=Masaki Kashiwara, Japanese Mathematician, Wins 2025 Abel Prize.
 author=Kenneth Chang,
 time = Wed, 26 Mar 2025 13:12:21 +0000
 [x] Received title=South Korea, World’s Largest ‘Baby Exporter,’ Admits to Adoption Fraud.
 author=Choe Sang-Hun,
 time = Wed, 26 Mar 2025 10:12:57 +0000
 [x] Received title=South Korea Wildfires Kill 26 and Destroy 1,000-Year-Old Temples.
 author=Jin Yu Young,
 time = Thu, 27 Mar 2025 04:00:28 +0000
 [x] Received title=Why These Islanders Hunt Dolphins.
 author=Prianka Srinivasan and Matthew Abbott,
 time = Wed, 26 Mar 2025 15:52:37 +0000
 [x] Received title=India Is on a Hiring Binge That Trump’s Tariffs Can’t Stop.
 author=Alex Travelli and Hari Kumar,
 time = Wed, 26 Mar 2025 07:04:28 +0000

 [*] Exiting.
